In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel, Matern, RationalQuadratic, ExpSineSquared, DotProduct
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import warnings
import pickle
import itertools
warnings.filterwarnings("ignore")


In [2]:
# Import data from 'Data/DATA_SOLMAT.xlsx'
doe = np.array(pd.read_excel('Data/DATA_SOLMAT.xlsx', engine='openpyxl',sheet_name = 'Normalized_table'))
data_exp = np.array(doe[:10])
X_exp = data_exp[:,:7]
Y_exp = data_exp[:,8:]

In [3]:
class GPUCB():

    def __init__(self, meshgrid, X, Y, beta=100.,delta = 0.1, dim = 2, pre_defined_gp = None):
        '''
        meshgrid: Output from np.methgrid.
        e.g. np.meshgrid(np.arange(-1, 1, 0.1), np.arange(-1, 1, 0.1)) for 2D space
        with |x_i| < 1 constraint.
        beta (optional): Hyper-parameter to tune the exploration-exploitation
        balance. If beta is large, it emphasizes the variance of the unexplored
        solution solution (i.e. larger curiosity)
        '''
        # 参考 https://github.com/tushuhei/gpucb/blob/master/gpucb.py        
        self.meshgrid = np.array(meshgrid)
        self.experimen_result= Y
        self.beta = beta
        self.iteration = 1
        self.dim = dim
        self.delta = delta
        self.acquired_pts = []
        self.x_only = []
        self.y_only = []  # for plotting
        if pre_defined_gp:
            self.gp = pre_defined_gp
        else:
            self.gp = GaussianProcessRegressor()
        self.X_grid = self.meshgrid.reshape(self.meshgrid.shape[0], -1).T
        self.mu, self.sigma = self.gp.predict(self.X_grid, return_std=True)
        self.X = []
        self.T = []
        
    def argmax_ucb(self):
        res = np.argmax(self.mu + self.sigma * np.sqrt(self.beta))
        return res

    def learn(self):
        print("shape of X_grid",self.X_grid.shape)
        self.beta = 2*np.log(self.X_grid.shape[1]*(self.iteration**2)*(np.pi**2)/(6*self.delta))
        grid_idx = self.argmax_ucb()
        self.sample(self.X_grid[grid_idx])
        self.gp.fit(self.X, self.T)
        self.mu, self.sigma = self.gp.predict(self.X_grid, return_std=True)
        self.iteration += 1    
    
    def sample(self, x):
        print("acquired point: ", x)
        t = self.experimen_result[tuple(x)]
        self.acquired_pts.append((x,t))
        self.y_only.append(t)
        self.x_only.append(x)
        self.X.append(x)
        self.T.append(t)

In [4]:
# temp 0-1 interval 0.2
# time interval 1/11
# pressure 1/18
# flow rate 1/9
# power 1/7
# distance 1/6
# ald_cycles 1/3
gp = GaussianProcessRegressor()
search_space = np.meshgrid(np.arange(0,1.01,0.2),np.arange(0,1.01,1/11),
                           np.arange(0,1.0001,1/18), np.arange(0,1.0001,1/9),
                           np.arange(0,1.0001,1/7), np.arange(0,1.0001,1/6),
                           np.arange(0,1.0001,1/3)
                          )
max_Y = np.max(Y_exp,axis=1)
gp.fit(X_exp,max_Y)
lookup = {tuple(X_exp[i]):Y_exp[i] for i in range(len(X_exp))}
print("The size of search space is: ", np.product([len(i) for i in [np.arange(0,1.01,0.2),np.arange(0,1.01,1/11),
                           np.arange(0,1.0001,1/18), np.arange(0,1.0001,1/9),
                           np.arange(0,1.0001,1/7), np.arange(0,1.0001,1/6),
                           np.arange(0,1.0001,1/3)]]))
test = GPUCB(search_space, X_exp, lookup, beta=0,delta = 0.01, dim = 7, 
             pre_defined_gp = gp)
test.learn()


The size of search space is:  3064320
shape of X_grid (3064320, 7)
acquired point:  [0.6        0.54545455 0.27777778 0.44444444 0.28571429 0.33333333
 0.33333333]


KeyError: (0.6000000000000001, 0.5454545454545454, 0.2777777777777778, 0.4444444444444444, 0.2857142857142857, 0.3333333333333333, 0.3333333333333333)

In [ ]:
np.arange(0,1.01,1/11)

In [ ]:
Y_exp